In [1]:
import os
import numpy as np
from PIL import Image
from sklearn.model_selection import train_test_split

In [2]:
# Função para carregar imagens de um diretório e redimensionar
def load_images_from_folder(folder, img_size, labels_dict=None, max_images=None, sort=False):
    images = []
    labels = []

    file_list = os.listdir(folder)
    # Ordenar os arquivos numericamente (por conta do submission file)
    if sort:
        file_list = sorted(file_list, key=lambda x: int(x.split('.')[0]))
    
    max_images = len(file_list) // 4

    for filename in file_list:
        if len(images) >= max_images:
            break
        if max_images and len(images) >= max_images:
            break
        if filename.endswith(".jpg"):
            img_path = os.path.join(folder, filename)
            img = Image.open(img_path).resize((img_size, img_size)).convert('RGB')
            img_array = np.array(img)
            images.append(img_array)
            # Etiquetar as imagens: 0 para gato, 1 para cachorro
            if labels_dict:
                labels.append(labels_dict[filename])
            elif 'cat' in filename:
                labels.append('cat')
            elif 'dog' in filename:
                labels.append('dog')
    return np.array(images), np.array(labels)

In [3]:
train_folder = './train'
submission_folder = './samples'
img_size = 64  # Redimensionar imagens para 64x64 pixels

# Carregar imagens de treino
X_train, y_train = load_images_from_folder(train_folder, img_size)

# Para o conjunto de teste, podemos simplesmente carregar as imagens sem etiquetas
# ou usar um conjunto de validação a partir dos dados de treinamento
X_train, X_test, y_train, y_test = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

X_submission, _ = load_images_from_folder(submission_folder, img_size, sort=True)

In [4]:
# Normalizar os dados
X_train = X_train.astype('float32') / 255.0
X_test = X_test.astype('float32') / 255.0
X_submission = X_submission.astype('float32') / 255.0

In [5]:
X_train_flat = X_train.reshape((X_train.shape[0], -1))
X_test_flat = X_test.reshape((X_test.shape[0], -1))
X_submission_flat = X_submission.reshape((X_submission.shape[0], -1))

print(X_train_flat.shape)
print(X_test_flat.shape)
print(X_submission_flat.shape)

(5000, 12288)
(1250, 12288)
(100, 12288)


In [6]:
from sklearn.decomposition import PCA

n_components = 500
pca = PCA(n_components=n_components)
X_train_pca = pca.fit_transform(X_train_flat)
X_test_pca = pca.transform(X_test_flat)
X_submission_pca = pca.transform(X_submission_flat)

In [7]:
from sklearn.linear_model import LogisticRegression
# Treinamento de um modelo Regressão Logistica
def train(X_train, y_train):
  model = LogisticRegression(max_iter=10000)
  model.fit(X_train, y_train)
  return model

model = train(X_train_pca, y_train)

In [8]:
from sklearn.metrics import accuracy_score, cohen_kappa_score, f1_score, confusion_matrix
import seaborn as sns

# Inferência e avaliação
def predict_and_evaluate(model, X_test, y_test):

    # Inferência
    y_pred = model.predict(X_test)

    # Métricas
    print('Acurácia:', accuracy_score(y_test, y_pred))
    print('F1 score:', f1_score(y_test, y_pred, average='weighted'))

    # Matriz de confusão
    conf_matrix = confusion_matrix(y_test, y_pred)

    # sns.heatmap(conf_matrix, annot=True, fmt='d', cmap="Blues", xticklabels=np.unique(y_test), yticklabels=np.unique(y_test))
    # plt.title('Matriz de Confusão')
    # plt.xlabel('Predito')
    # plt.ylabel('Verdadeiro')
    # plt.show()

print('Resultados de Treino')
predict_and_evaluate(model, X_train_pca, y_train)
print('Resultados de Teste')
predict_and_evaluate(model, X_test_pca, y_test)

Resultados de Treino
Acurácia: 0.6858
F1 score: 0.685790762357048
Resultados de Teste
Acurácia: 0.6024
F1 score: 0.6023172661064425


In [9]:
from skimage.feature import hog
# Extração de características utilizando HOG
def extract_hog_features(images):
    hog_features = []
    hog_images = []
    for image in images:
        features, img = hog(image, orientations=9,      # o número de direções angulares distintas para as quais os gradientes são calculados
                              pixels_per_cell=(8, 8),   # tamanho (em pixels) de cada célula na qual a imagem é dividida para calcular o histograma de gradientes orientados
                              cells_per_block=(2, 2),   # número de células em cada bloco. Os blocos são usados para normalizar os histogramas de gradientes dentro de células, melhorando a robustez a variações de iluminação e contraste.
                              visualize=True,           # se a imagem HOG (uma representação visual das características HOG) deve ser retornada junto com o vetor de características.
                              channel_axis=-1)          # índice do eixo do canal na imagem de entrada.
        hog_features.append(features)
        hog_images.append(img)
    return np.array(hog_features), hog_images

# Extraindo características HOG dos dados de treinamento e teste
X_train_hog, train_hog_images = extract_hog_features(X_train)
X_test_hog, _ = extract_hog_features(X_test)
X_submission_hog, _ = extract_hog_features(X_submission)

In [10]:
model = train(X_train_hog, y_train)

print('Resultados de Treino')
predict_and_evaluate(model, X_train_hog, y_train)
print('Resultados de Teste')
predict_and_evaluate(model, X_test_hog, y_test)

Resultados de Treino
Acurácia: 0.8116
F1 score: 0.811580765539811
Resultados de Teste
Acurácia: 0.6728
F1 score: 0.6727319151660663


# Exercício: Concatene os componentes principais e as features extraídas com HOG e treine novamente o modelo

In [11]:
print(X_train_hog.shape)
print(X_train_pca.shape)

(5000, 1764)
(5000, 500)


In [12]:
# Concatenar as características HOG e PCA
X_train_concat = np.hstack((X_train_hog, X_train_pca))
X_test_concat = np.hstack((X_test_hog, X_test_pca))
X_submission_concat = np.hstack((X_submission_hog, X_submission_pca))

In [13]:
print(X_train_concat.shape)

(5000, 2264)


In [14]:
model = train(X_train_concat, y_train)

print('Resultados de Treino')
predict_and_evaluate(model, X_train_concat, y_train)
print('Resultados de Teste')
predict_and_evaluate(model, X_test_concat, y_test)

Resultados de Treino
Acurácia: 0.8598
F1 score: 0.8598000729044929
Resultados de Teste
Acurácia: 0.684
F1 score: 0.6838127639512636


# Exercício: Inferir imagens para submissão no kaggle

In [15]:
y_pred = model.predict(X_submission_concat)

In [16]:
import pandas as pd
results = pd.DataFrame()
results['id'] = list(range(1, y_pred.shape[0] + 1))
results['label'] = y_pred
results.head()

,id,label
0,1,cat
1,2,dog
2,3,dog
3,4,dog
4,5,cat


In [21]:
y_pred = model.predict(X_submission_concat)

import pandas as pd
from datetime import datetime

results = pd.DataFrame()
results['id'] = list(range(1, y_pred.shape[0] + 1))
results['label'] = y_pred
results.head()

# Generate the current timestamp
timestamp = datetime.now().strftime('%Y-%m-%d_%H-%M')

# Generate the file name with the timestamp
file_name = f'results_{timestamp}.csv'

# Save the DataFrame to a CSV file
results.to_csv(file_name, index=False)

# Display the first few rows of the DataFrame
results.head()


SyntaxError: invalid syntax (3471390363.py, line 1)

## Tuning de parâmetros

In [18]:
import optuna
from sklearn.ensemble import RandomForestClassifier

# Variáveis para armazenar o melhor modelo e a melhor métrica
best_model = None
best_accuracy = 0.0

def objective(trial):

    # Definir o espaço de busca para os hiperparâmetros
    n_estimators = trial.suggest_int('n_estimators', 10, 200)
    max_depth = trial.suggest_int('max_depth', 5, 40)
    min_samples_leaf = trial.suggest_int('min_samples_leaf', 1, 30)

    # Cria modelo
    model = RandomForestClassifier(n_estimators=n_estimators,
                                   max_depth=max_depth,
                                   min_samples_leaf=min_samples_leaf,
                                   random_state=42)

    model.fit(X_train_concat, y_train)  # Treina
    y_pred = model.predict(X_test_concat)  # Infere
    accuracy = accuracy_score(y_test, y_pred)  # Calcula métrica

    # Atualize o melhor modelo se necessário
    global best_model, best_accuracy
    if accuracy > best_accuracy:
        best_model = model
        best_accuracy = accuracy

    return accuracy

# Cria estudo a ser otimizado
# study = optuna.create_study(direction='maximize')
# study.optimize(objective, n_trials=100, n_jobs=-1)

# # Imprime a evolução dos modelos ao final da otimização
# print(f'Melhor solução:')
# trial = study.best_trial

# print(f'Valor: {trial.value}')
# print(f'Parâmetros: ')
# for key, value in trial.params.items():
#     print(f'    {key}: {value}')

/home/teixeira/code/computer-vision-master/Técnicas Tradicionais de Classificação de Imagens/dogs-vs-cats/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [20]:
import optuna
from xgboost import XGBClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score

# Codificar os rótulos
label_encoder = LabelEncoder()
y_train_encoded = label_encoder.fit_transform(y_train)
y_test_encoded = label_encoder.transform(y_test)

# Variáveis para armazenar o melhor modelo e a melhor métrica
best_model = None
best_accuracy = 0.0

def objective(trial):
    # Definir o espaço de busca para os hiperparâmetros
    n_estimators = trial.suggest_int('n_estimators', 10, 200)
    max_depth = trial.suggest_int('max_depth', 5, 40)
    learning_rate = trial.suggest_float('learning_rate', 0.01, 0.3, log=True)


    # Cria modelo
    model = XGBClassifier(
        n_estimators=n_estimators,
        max_depth=max_depth,
        learning_rate=learning_rate
    )

    model.fit(X_train_concat, y_train_encoded)  # Treina
    y_pred = model.predict(X_test_concat)  # Infere
    accuracy = accuracy_score(y_test_encoded, y_pred)  # Calcula métrica

    # Atualize o melhor modelo se necessário
    global best_model, best_accuracy
    if accuracy > best_accuracy:
        best_model = model
        best_accuracy = accuracy

    return accuracy

# Cria estudo a ser otimizado
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=300)

# Imprime a evolução dos modelos ao final da otimização
print(f'Melhor solução:')
trial = study.best_trial

print(f'Valor: {trial.value}')
print(f'Parâmetros: ')
for key, value in trial.params.items():
    print(f'    {key}: {value}')


[I 2024-08-07 01:49:00,077] A new study created in memory with name: no-name-2bcd1853-3b89-4ca5-8751-4d5ebd947add
[I 2024-08-07 01:51:08,334] Trial 0 finished with value: 0.7056 and parameters: {'n_estimators': 126, 'max_depth': 29, 'learning_rate': 0.013428133496533382}. Best is trial 0 with value: 0.7056.
[I 2024-08-07 01:52:50,648] Trial 1 finished with value: 0.7096 and parameters: {'n_estimators': 114, 'max_depth': 23, 'learning_rate': 0.02757334649604389}. Best is trial 1 with value: 0.7096.
[I 2024-08-07 01:53:58,948] Trial 2 finished with value: 0.7336 and parameters: {'n_estimators': 160, 'max_depth': 12, 'learning_rate': 0.08510748626525892}. Best is trial 2 with value: 0.7336.
[I 2024-08-07 01:54:07,616] Trial 3 finished with value: 0.7064 and parameters: {'n_estimators': 73, 'max_depth': 5, 'learning_rate': 0.1049343633470188}. Best is trial 2 with value: 0.7336.
[I 2024-08-07 01:54:32,766] Trial 4 finished with value: 0.7184 and parameters: {'n_estimators': 90, 'max_depth'

Melhor solução:
Valor: 0.7536
Parâmetros: 
    n_estimators: 158
    max_depth: 9
    learning_rate: 0.08508346541012761


In [22]:
y_pred = model.predict(X_submission_concat)

import pandas as pd
from datetime import datetime

results = pd.DataFrame()
results['id'] = list(range(1, y_pred.shape[0] + 1))
results['label'] = y_pred
results.head()

# Generate the current timestamp
timestamp = datetime.now().strftime('%Y-%m-%d_%H-%M')

# Generate the file name with the timestamp
file_name = f'results_{timestamp}.csv'

# Save the DataFrame to a CSV file
results.to_csv(file_name, index=False)

# Display the first few rows of the DataFrame
results.head()

,id,label
0,1,cat
1,2,dog
2,3,dog
3,4,dog
4,5,cat
